In [1]:
import pandas as pd 
import numpy as np
cities = pd.read_csv("data/cities.csv") 
providers = pd.read_csv("data/providers.csv") 
stations = pd.read_csv("data/stations.csv") 
ticket_data = pd.read_csv("data/ticket_data.csv")
import warnings
warnings.filterwarnings('ignore')

In [2]:
cities.head()

,id,local_name,unique_name,latitude,longitude,population
0,5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
1,76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
2,81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
3,259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
4,11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


In [3]:
providers.head()

,id,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
0,9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
1,10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
2,11,3,NaN,infobus,Infobus,False,False,False,False,bus
3,12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
4,13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


In [4]:
stations.head()

,id,unique_name,latitude,longitude
0,1,Aalen (Stuttgarter Straße),48.835296,10.092956
1,2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
2,3,Aéroport CDG,49.009900,2.559310
3,4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
4,5,Aéroport de Dresden,51.123604,13.764737


In [5]:
ticket_data.head()

,id,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
0,6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
1,6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
2,6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
3,6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
4,6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


In [6]:
ticket_data.isnull().sum()

id                     0
company                0
o_station          41441
d_station          41441
departure_ts           0
arrival_ts             0
price_in_cents         0
search_ts              0
middle_stations    41441
other_companies    41441
o_city                 0
d_city                 0
dtype: int64

In [7]:
ticket_data['arrival_ts'] = pd.to_datetime(ticket_data['arrival_ts'])
ticket_data['departure_ts'] = pd.to_datetime(ticket_data['departure_ts'])

In [8]:
ticket_data['duration'] = ticket_data['arrival_ts'] - ticket_data['departure_ts']

In [9]:
ticket_data['duration_numeric'] = ticket_data['duration'].values.astype(np.int64)

In [10]:
ticket_data_price_desc = ticket_data.groupby(['o_city', 'd_city']).agg({'price_in_cents': ['mean', 'min', 'max']})

In [11]:
print(ticket_data_price_desc)

              price_in_cents              
                        mean    min    max
o_city d_city                             
5      23       20320.000000  18600  22000
6      227      11755.000000   9860  13650
       504       4042.666667   2000   8920
       628       2797.500000   2600   3190
       845        864.626866    700   2420
...                      ...    ...    ...
11938  126       4204.861111   3000   5350
12124  1064      6950.000000   6000   7900
12166  857       5300.000000   5300   5300
12190  639        688.888889    600    850
       8937      9800.000000   9800   9800

[1437 rows x 3 columns]


Un trajet est constitué d'un point de depart et d'un point d'arrivée

In [12]:

ticket_data_time_desc = ticket_data.groupby(['o_city', 'd_city']).agg({'duration_numeric': ['mean', 'min', 'max']})

In [13]:
#supprimer la meta ligne generer par l'operateur agg
ticket_data_time_desc.columns = ticket_data_time_desc.columns.droplevel(0)
#convertir en timedelta plus parlant
ticket_data_time_desc['mean'] = pd.to_timedelta(ticket_data_time_desc['mean'])
ticket_data_time_desc['min'] = pd.to_timedelta(ticket_data_time_desc['min'])
ticket_data_time_desc['max'] = pd.to_timedelta(ticket_data_time_desc['max'])

In [14]:
print(ticket_data_time_desc)

                                   mean             min             max
o_city d_city                                                          
5      23               0 days 10:18:48 0 days 08:53:00 0 days 15:54:00
6      227              0 days 13:42:30 0 days 12:24:00 0 days 15:01:00
       504              0 days 08:17:24 0 days 05:36:00 0 days 12:20:00
       628              0 days 12:10:00 0 days 09:40:00 0 days 14:30:00
       845    0 days 01:19:54.626865671 0 days 01:00:00 0 days 04:11:00
...                                 ...             ...             ...
11938  126    0 days 07:25:16.666666666 0 days 05:30:00 1 days 11:20:00
12124  1064             0 days 17:10:00 0 days 11:10:00 0 days 23:10:00
12166  857              0 days 21:55:00 0 days 21:55:00 0 days 21:55:00
12190  639    0 days 01:28:53.333333333 0 days 01:10:00 0 days 02:40:00
       8937             0 days 09:10:00 0 days 09:10:00 0 days 09:10:00

[1437 rows x 3 columns]


In [15]:
ticket_providers = pd.merge(ticket_data, providers, how='right', left_on=['company'], right_on=['id'])

In [16]:
data_part_1 = pd.merge(ticket_providers, cities[['id', 'latitude', 'longitude']], how='right', left_on=['o_city'], right_on=['id'],suffixes = ("","_o_city"))

In [17]:
data_with_all_coord = pd.merge(data_part_1, cities[['id', 'latitude', 'longitude']], how='right', left_on=['d_city'], right_on=['id'],suffixes = ("","_d_city"))

In [18]:
dat=data_with_all_coord[['id','o_city','d_city','duration','price_in_cents','longitude','latitude', 'longitude_d_city','latitude_d_city','transport_type']]
dat_filter = dat[['id','o_city','d_city','duration','price_in_cents','longitude','latitude', 'longitude_d_city','latitude_d_city','transport_type']]


In [19]:
def haversine_vectorize(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    newlon = lon2 - lon1
    newlat = lat2 - lat1
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
    dist = 2 * np.arcsin(np.sqrt(haver_formula))
    km = 6367 * dist
    return km

In [20]:
dat['distance'] = haversine_vectorize(data_with_all_coord['longitude'],data_with_all_coord['latitude'],data_with_all_coord['longitude_d_city'],data_with_all_coord['latitude_d_city'])

In [21]:
dat

,id,o_city,d_city,duration,price_in_cents,longitude,latitude,longitude_d_city,latitude_d_city,transport_type,distance
0,NaN,NaN,NaN,NaT,NaN,NaN,NaN,11.876761,45.406435,NaN,NaN
1,639.0,639.0,76.0,0 days 02:40:00,1450.0,2.889866,42.697118,2.173404,41.385064,carpooling,157.333773
2,639.0,639.0,76.0,0 days 02:40:00,1700.0,2.889866,42.697118,2.173404,41.385064,carpooling,157.333773
3,639.0,639.0,76.0,0 days 02:40:00,1450.0,2.889866,42.697118,2.173404,41.385064,carpooling,157.333773
4,639.0,639.0,76.0,0 days 02:50:00,1450.0,2.889866,42.697118,2.173404,41.385064,carpooling,157.333773
...,...,...,...,...,...,...,...,...,...,...,...
81871,467.0,467.0,11144.0,0 days 02:05:00,1906.0,-0.127758,51.507351,-0.137400,50.822000,bus,76.162660
81872,467.0,467.0,11144.0,0 days 03:45:00,1906.0,-0.127758,51.507351,-0.137400,50.822000,bus,76.162660
81873,467.0,467.0,11144.0,0 days 03:26:00,1906.0,-0.127758,51.507351,-0.137400,50.822000,bus,76.162660
81874,NaN,NaN,NaN,NaT,NaN,NaN,NaN,-0.462100,39.493000,NaN,NaN


In [22]:
ticket_data_time_desc = dat.groupby(['o_city', 'd_city','transport_type']).agg({'price_in_cents': ['mean', 'min', 'max']})

In [23]:
dat['duration_numeric'] = dat['duration'].values.astype(np.int64)
ticket_data_time_desc = dat.groupby(['o_city', 'd_city','transport_type']).agg({'duration_numeric': ['mean', 'min', 'max']})

In [24]:
dat['duration_numeric'] = dat['duration'].values.astype(np.int64)
ticket_data_time_desc = dat.groupby(['o_city', 'd_city','transport_type']).agg({'duration_numeric': ['mean', 'min', 'max']})

In [33]:
test = dat.groupby([pd.cut(dat["distance"], np.arange(0, 2800, 200)), 'transport_type']).agg({'distance': ['mean', 'min', 'max'],'price_in_cents':['mean', 'min', 'max']})

In [34]:
print(test)

                                distance                            \
                                    mean          min          max   
distance     transport_type                                          
(0, 200]     bus              120.064379    27.361184   199.980193   
             carpooling       132.191268    18.901367   199.980193   
             train            134.461709    19.271988   199.980193   
(200, 400]   bus              297.884161   200.142144   399.797444   
             carpooling       298.691069   200.142144   399.797444   
             train            319.808086   200.142144   399.797444   
(400, 600]   bus              507.790252   403.306366   599.801228   
             carpooling       503.670651   400.073660   599.801228   
             train            492.523703   404.565456   599.801228   
(600, 800]   bus              691.131882   600.168517   798.006311   
             carpooling       674.180441   601.528374   787.891705   
             train  